In [ ]:
%use roboquant
Welcome()

# Roboquant

Roboquant is framework for algoritmic trading written in Kotlin. It is fast, flexible and tries to be easy to use. It is aimed at serious retail traders as well as smaller prop shops. Some key features include:

...



**Warning**: right now roboquant is still in beta phase. So we recommend to be carefull when using the IEX Cloud real data API if you treasure your credits. A small bug in roboquant or your own code can quickly burn a lot of credits.



# Feed
We start with setting up the data we want to use for our back-test. In this case we are going to use historic price data from IEX Cloud. By default the IEXFeed uses the sandbox environment. This is a good option for testing if the setup works as expected since any API calls don't cost you credits. However the data returned is randomized, so not very useful for real use-cases. But for this sample notebook it is sufficient.


In [ ]:
import org.roboquant.feeds.iex.*

In [ ]:
val publishableToken = Config.getProperty("IEX_PUBLISHABLE_TOKEN")!!
val feed = IEXFeed(publishableToken)

// If you want to use the real data API, make sure that you use the correct token and then invoke
// val feed = IEXFeed(publishableToken, sandbox=false)

# Getting the data
Now we are going to retrieve the prices for the assets we are interested in for our back-test.  The retrievePriceBar method also allows specify a time range you want to get the data from. Options include: max, today, 1d, 5d, 5dm, 1m, 1mm, 3m, 6m, ytd, 1y, 2y, 5y. But we go with the default that is 5 years of historic data.

After we have called the retreivePriceBar method the resulting pricebars are stored in local memory. So there remaining code doesn't call the API again and no additional credits are wasted.

In [ ]:
feed.retrievePriceBar("AAPL", "IBM", "FB")

# Lets inspect what prices we retrieved.
Please note that if you used the sandbox environment this won't be the real prices for the last five years.  

In [ ]:
CandlestickChart.fromFeed(feed, feed.assets.first())

# Run the Back Test
We are now ready to run the back test. Roboquant is optimized to process large amount of data, so 5 year of data for just two assets doesn't take long. On a modern CPU the test is run in less than 1 second.

In [ ]:
val strategy = EMACrossover(10, 30)
val metric = AccountSummary()
val roboquant =  Roboquant(strategy, metric)

In [ ]:
roboquant.run(feed)

In [ ]:
val account = roboquant.broker.account
account.summary()

In [ ]:
account.trades.summary()

In [ ]:
TradeChart(account.trades)

# Saving API calls (and credits)

If you want to continue at a later time and call it a day, you can save the feed to an Avro file and later use this file to re-create the feed. This will save you credits since you don't have to call the APIs again and even enable you to work offline if desired.

Alternatively you could also use [Cloud Cache](https://iexcloud.io/cloud-cache/) that is offered as a service by IEX Cloud.

In [ ]:
val gen = AvroGenerator()
gen.capture(feed, "iexcloud.avro")

Then at a later stage you can easily re-create the feed by using the AvroFeed with the saved file 

In [ ]:
val feed = AvroFeed("iexcloud.avro")

In [ ]:
feed.assets

# Thanks
Under the hood roboquant uses the [IEXTrading4J](https://github.com/WojciechZankowski/iextrading4j) library to interact with IEX Cloud. This is a great and solid library with many features and I highly recommend to check it out. 